# Fases de um Projeto de Machine Learning
- 1 - Definição do Problema
- 2 - Preparação dos Dados
- 3 - Crie o Modelo
- 4 - Deploy - Colocar em Produção



## 1. Definição do Problema
Uma empresa que oferece empréstimo a pessoas físicas, necessita saber, com base nos dados de seus clientes, quais novos clientes poderão ou não ter acesso ao crédito solicitado.

## 2. Preparação dos Dados

In [0]:
# Importando as Bibliotecas
import os
import re
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

### Carregando a Fonte de Dados (Análise de Crédito)

In [0]:
# Importando arquivo .csv
data = pd.read_csv('loan.csv')

In [0]:
# Verifica as primeiras 5 linhas
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [0]:
# Informações sobre colunas
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
Loan_ID              614 non-null object
Gender               601 non-null object
Married              611 non-null object
Dependents           599 non-null object
Education            614 non-null object
Self_Employed        582 non-null object
ApplicantIncome      614 non-null int64
CoapplicantIncome    614 non-null float64
LoanAmount           592 non-null float64
Loan_Amount_Term     600 non-null float64
Credit_History       564 non-null float64
Property_Area        614 non-null object
Loan_Status          614 non-null object
dtypes: float64(4), int64(1), object(8)
memory usage: 62.4+ KB


In [0]:
# Verificando o balanceamento dos Labels (Y - Aprovado, N - Negado)
data.Loan_Status.value_counts()

Y    422
N    192
Name: Loan_Status, dtype: int64

In [0]:
# Reduzindo o valor de Aprovados para ficar equilibrado
data2 = data[data.Loan_Status=='Y'].sample(200)

In [0]:
data = data2.append(data[data.Loan_Status=='N'].sample(192))

In [0]:
# Verificando novamente (agora tá mais equilibrado)
data.Loan_Status.value_counts()

Y    200
N    192
Name: Loan_Status, dtype: int64

### Checando Missing Values (Valores nulos)

In [0]:
data2.isnull().sum()

Loan_ID               0
Gender                3
Married               2
Dependents            6
Education             0
Self_Employed        10
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            7
Loan_Amount_Term      3
Credit_History       18
Property_Area         0
Loan_Status           0
dtype: int64

Preechendo Missing Values:

- `Dependents`: Assumindo o valor majoritário da coluna.
- `Self_Employed`: Assumindo o valor majoritário da coluna.
- `Loan_Amount_Term`: Preenchendo com o valor médio da coluna.
- `Credit_History`: Assumindo o valor majoritário da coluna.
- `Married`: Assumindo o valor majoritário da coluna.
- `Gender`: Assumindo o valor majoritário da coluna.

In [0]:
data['Gender'] = data['Gender'].fillna('Male')

In [0]:
data['Married'] = data['Married'].fillna('No')

In [0]:
data['Dependents'] = data['Dependents'].fillna('0')

In [0]:
data['Self_Employed'] = data['Self_Employed'].fillna('No')

In [0]:
data['LoanAmount'] = data['LoanAmount'].fillna(data['LoanAmount'].mean())

In [0]:
data['Credit_History'] = data['Credit_History'].fillna(1.0)

In [0]:
data['Loan_Amount_Term'] = data['Loan_Amount_Term'].fillna(data['Loan_Amount_Term'].mean())

#### Checando novamente Missing Values

In [0]:
data.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

**Transformando dados categóricos**

Várias colunas do dataframe são categóricas, precisamos transforma-las, são elas: `Gender`, `Married`, `Education`, `Self_Employed`, `Dependents` e `Loan_Status` columns.

In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
gender_values = {'Female' : 0, 'Male' : 1} 
married_values = {'No' : 0, 'Yes' : 1}
education_values = {'Graduate' : 0, 'Not Graduate' : 1}
employed_values = {'No' : 0, 'Yes' : 1}
dependent_values = {'3+': 3, '0': 0, '2': 2, '1': 1}
loan_values = {'Y':1,'N':0}
data.replace({'Gender': gender_values,
                 'Married': married_values, 
                 'Education': education_values,
                 'Self_Employed': employed_values, 
                 'Dependents': dependent_values,
                 'Loan_Status': loan_values
                }, inplace=True)

In [0]:
# Apagando colunas que não agrega valor ao modelo
data.drop(['Loan_ID','CoapplicantIncome','Loan_Amount_Term','Credit_History','Property_Area'],axis=1,inplace=True)

In [0]:
# Verificando
data.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,LoanAmount,Loan_Status
316,1,1,2,0,0,3717,120.000000,1
207,1,0,0,1,0,3975,55.000000,1
103,1,1,0,0,0,4652,147.125668,1
79,1,1,3,1,1,3333,130.000000,1
491,1,1,1,1,0,5333,186.000000,1


## 3. Crie o Modelo

In [0]:
# O classificador escolhido para o estudo foi o Random Forest
# - O foco não foi os melhores resultados, e sim mostrar o fluxo completo
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [0]:
# Instanciando o Classificador
clf_rf = RandomForestClassifier(n_estimators=100,min_samples_split=2)

In [0]:
# Dividindo o conjunto de dados (Valores e Rótulos)
X_train = data.drop('Loan_Status',axis=1)
y = data['Loan_Status']

In [0]:
# Divindo o conjunto em Treinamento e Teste (tamanho do teste: 20% do conjunto)
X_treino, X_teste, y_treino, y_teste = train_test_split(X_train, y,test_size=0.20,random_state=42)

In [0]:
# Treinando o modelo
clf_rf.fit(X_treino,y_treino)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

### Métricas de Validação ###

In [0]:
# Importando bibliotecas de métricas
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [0]:
# Usando crosstab para visualizar as classificações
# - O foco do estudo, não foi os melhores resultados, e sim mostrar o fluxo completo
print (pd.crosstab(y_teste, clf_rf.predict(X_teste), rownames=['Real'], colnames=['Predito'], margins=True), '')

(Predito   0   1  All
Real                
0        17  11   28
1        25  26   51
All      42  37   79, '')


In [0]:
# Relatório de Classificação
# - O foco do estudo, não foi os melhores resultados, e sim mostrar o fluxo completo
# -- esses números podem melhorar
print (metrics.classification_report(y_teste,clf_rf.predict(X_teste)))

              precision    recall  f1-score   support

           0       0.40      0.61      0.49        28
           1       0.70      0.51      0.59        51

   micro avg       0.54      0.54      0.54        79
   macro avg       0.55      0.56      0.54        79
weighted avg       0.60      0.54      0.55        79



## 4. Coloque em Produção

### Persistindo o modelo de Machine Learning para o disco. ###

In [0]:
# Usando o joblib
from sklearn.externals import joblib

#### Persistindo o melhor modelo em disco.

In [0]:
joblib.dump(clf_rf, 'model/model.pkl')

['/content/drive/My Drive/0. Business/2. Consultoria em Dados/2. IA, ML/0. Scripts, Exemplos, Cursos/Scripts exemplos/Deploy em Produc\xcc\xa7a\xcc\x83o/model.pkl']

#### Carregando o modelo a partir do disco para a memória.

In [0]:
# Carregando do Disco para a Memória
model = joblib.load('model.pkl')

#### Verificando os atributos do modelo.

In [0]:
print("Atributos do Modelo:\n\nClasses:{}\n\nEstimators:{}\n\nParametros:{}".format(model.classes_,model.n_estimators,model.base_estimator))

Atributos do Modelo:

Classes:[0 1]

Estimators:100

Parametros:DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')


**Verificando o Dataset final gerado.**

In [0]:
X_train.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,LoanAmount
264,1,0,0,0,1,5166,128.0
566,1,0,0,0,0,3333,70.0
419,1,1,0,0,0,3246,138.0
512,1,1,2,0,0,3283,148.0
442,1,0,3,1,0,4707,148.0


**Teste de Classificação.**

In [0]:
teste = np.array([[1,1,3,0,0,9504,275.0]])

In [0]:
model.predict(teste)

array([0])

**Probabilidades de Classes.**

In [0]:
model.predict_proba(teste)

array([[0.56, 0.44]])

### Publicando na Web
Estrutura do Diretório
- Model
 - modelo gerado acima deve ficar nessa pasta (model.pkl)
- static
 - arquivos auxiliares da página
- templates
 - página principal (template.html)
- *srv.py*
 - Renderizando a página usando Flask

#### Publicando...
No terminal, execute:

```
C:\Python>python srv.py

 * Serving Flask app "srv" (lazy loading)
 * Environment: production
   WARNING: This is a development server. Do not use it in a production deployment.
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://localhost:5500/ (Press CTRL+C to quit)

```

##### Página Publicada
![alt text](https://pessoalex.files.wordpress.com/2020/02/producao-e1581945097714.png)





## Referências
- Fases do Projeto ([Mauro Filho](http://cursodedatascience.com/))
- Base ([Minerando Dados](https://minerandodados.com.br/))
- Classificador: Random Forest Classifier ([Link](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html))
- Model Persistence ([joblib](https://scikit-learn.org/stable/modules/model_persistence.html))
- Modelo simples de Desenvolvimento Web ([Flask](https://www.treinaweb.com.br/blog/o-que-e-flask/))
- Alex Souza ([Blog](https://pessoalex.wordpress.com/))
